In [1]:
#Import
import pandas as pd
import numpy as np
import torch
import re
from transformers import pipeline

In [2]:
#Sentiment Pipeline
model_path = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
sentiment_task = pipeline("sentiment-analysis", model=model_path, tokenizer=model_path)

In [3]:
#Test Sentiment Analyse
print("Ich liebe dich: " + str(sentiment_task("Ich liebe dich")))
print("Ich liebe dich nicht: " + str(sentiment_task("Ich liebe dich nicht")))
print("Ich hasse dich: " + str(sentiment_task("Ich hasse dich")))
print("I love you: " + str(sentiment_task("I love you")))
print("I hate you: " + str(sentiment_task("I hate you")))

Ich liebe dich: [{'label': 'Positive', 'score': 0.866546094417572}]
Ich liebe dich nicht: [{'label': 'Negative', 'score': 0.9222964644432068}]
Ich hasse dich: [{'label': 'Negative', 'score': 0.9443648457527161}]
I love you: [{'label': 'Positive', 'score': 0.7866936326026917}]
I hate you: [{'label': 'Negative', 'score': 0.9430438280105591}]


In [9]:
#Import Tweets
df = pd.read_csv('tweets_from_accounts.csv', lineterminator='\n')

#For Testing Purposes shorten to 3 Tweets
df = df.head(3)

#For Oversight delete unused collums
df = df[["id", "like_count", "text"]]
df

,id,like_count,text
0,1526540001995022337,158,"Spread Love, not CO2! 🏳️‍🌈\n#IDAHOBIT2022 #Yes..."
1,1526536363755151360,175,Quote of the day: Queer denken statt Querdenke...
2,1526534237972746242,274,Was bedeutet queer?\n\nQueer ist eine positive...


In [17]:
#Spalte als String umwandeln
df["text"] = df["text"].astype('string')
df

,id,like_count,text
0,1526540001995022337,158,"Spread Love, not CO2! 🏳️‍🌈 #IDAHOBIT2022 #YesT..."
1,1526536363755151360,175,Quote of the day: Queer denken statt Querdenke...
2,1526534237972746242,274,Was bedeutet queer? Queer ist eine positive S...


In [188]:
#Delete Username
#df["text"] = re.sub("@[A-Za-z0-9_]+","", df["text"])

#Delete Hashtags
#df["text"] = re.sub("#[A-Za-z0-9_]+","", df["text"])

#Delete Links
#df["text"] = re.sub(r"http\S+","", df["text"])
#df["text"] = re.sub(r"www.\S+","", df["text"])

In [20]:
#New Collumn Sentiment
df.assign(sentiment = sentiment_task(df["text"].to_list()))

,id,like_count,text,sentiment
0,1526540001995022337,158,"Spread Love, not CO2! 🏳️‍🌈 #IDAHOBIT2022 #YesT...","{'label': 'Neutral', 'score': 0.4576849937438965}"
1,1526536363755151360,175,Quote of the day: Queer denken statt Querdenke...,"{'label': 'Positive', 'score': 0.5055170655250..."
2,1526534237972746242,274,Was bedeutet queer? Queer ist eine positive S...,"{'label': 'Neutral', 'score': 0.8638097047805786}"
